<a href="https://colab.research.google.com/github/Irina-Bogdanova-bioinformatician/Bioinformatics_Workshop_BI_2022/blob/Ampicillin_resistance_research/Ampicillin_resistance_research/Ampicillin_resistance_research_Lab_Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# What causes ampicillin resistance? Lab Notebook

This journal was kept during the implementation of a project dedicated to finding the cause of ampicillin resistance in E. coli


## Downloading data, manual inspecting

**Get a reference sequence in fasta format (NC_000913.3 E. coli str. K-12 substr. MG1655, complete genome) and an annotation in .gff format**

In [1]:
%%shell

wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gff.gz

--2022-10-26 17:08:48--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.229, 165.112.9.230, 2607:f220:41f:250::228, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1379902 (1.3M) [application/x-gzip]
Saving to: ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’

GCF_000005845.2_ASM 100%[===================>]   1.32M  3.45MB/s    in 0.4s    

2022-10-26 17:08:49 (3.45 MB/s) - ‘GCF_000005845.2_ASM584v2_genomic.fna.gz’ saved [1379902/1379902]

--2022-10-26 17:08:49--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/005/845/GCF_000005845.2_ASM584v2/GCF_000005845.2_ASM584v2_genomic.gff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.229, 165.112.9.230, 2607:f220:41f:250::228, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|

**Get raw Illumina sequencing reads (paired) from shotgun sequencing of an E. coli strain that is resistant to the antibiotic ampicillin**




In [3]:
# %%shell
# wget https://figshare.com/ndownloader/files/23769689
# wget https://figshare.com/ndownloader/files/23769692
"""
Downloading files using this way resulted in getting corrupted files, so the files were downloaded via browser 
(https://figshare.com/articles/dataset/amp_res_2_fastq_zip/10006541/3)
"""


'\nDownloading files using this way resulted in getting corrupted files, so the files were downloaded via browser \n(https://figshare.com/articles/dataset/amp_res_2_fastq_zip/10006541/3)\n'

**Inspect raw sequencing data manually**

In [4]:
!zcat amp_res_1.fastq.gz | head -20

@SRR1363257.37 GWZHISEQ01:153:C1W31ACXX:5:1101:14027:2198 length=101
GGTTGCAGATTCGCAGTGTCGCTGTTCCAGCGCATCACATCTTTGATGTTCACGCCGTGGCGTTTAGCAATGCTTGAAAGCGAATCGCCTTTGCCCACACG
+
@?:=:;DBFADH;CAECEE@@E:FFHGAE4?C?DE<BFGEC>?>FHE4BFFIIFHIBABEECA83;>>@>@CCCDC9@@CC08<@?@BB@9:CC#######
@SRR1363257.46 GWZHISEQ01:153:C1W31ACXX:5:1101:19721:2155 length=101
GTATGAGGTTTTGCTGCATTCTCTGNGCGAATATTAACTCCNTNNNNNTTATAGTTCAAAGCAAGTACCTGTCTCTTATACACATCTCCGAGCCCACGAGC
+
@@<?=D?D==?<AFGDF+AIHEACH#22<:?E8??:9??GG#0#####000;CF=C)4.==CA@@@)=7?C7?E37;3@>;;(.;>AB#############
@SRR1363257.77 GWZHISEQ01:153:C1W31ACXX:5:1101:5069:2307 length=101
GCTTCTCTTAACTGAGGTCACCATCATGCCGTTAAGTCCCTACCTCTCTTTTGCCGGTAACTGTTCCGCCGCGATTGCCTTTTATCTGTCTCTTATACACC
+
??<DBD;4C2=<BB>:AC;<CF<CE@FE9@E1C@891CD*9:?:3D@DD4?D<DD:0;@A=AEIDDA##################################
@SRR1363257.78 GWZHISEQ01:153:C1W31ACXX:5:1101:5178:2440 length=101
GCATAAGGACGATCGCTCCAGAGTAAAATAAATACGCGCATGTGATACTCACAATACCAATGGTGAAGTTACGGGACTTAAACAAACTGAGATCAAGAATC
+
CCCF

In [5]:
!zcat amp_res_2.fastq.gz | head -20

@SRR1363257.37 GWZHISEQ01:153:C1W31ACXX:5:1101:14027:2198 length=101
GATCTAAGCTGAAGCCAGGCCAAAGTTTGACGATTGGTGCAGGCAGTAGCGCACAGCGACTGGCAAACAACAGCGATAGCATTACGTATCGTGTGCGCAAA
+
???BDB:DFHBFD@9;;+A;AFGH;ABHFHHGE@9:B:??@D>@;F?D8<<F8AA9EHHD8'..;5?A?A992(',(59CC3@C>22::A238+2>B<>B<
@SRR1363257.46 GWZHISEQ01:153:C1W31ACXX:5:1101:19721:2155 length=101
GTACTTGCTTTGNACTATAATATGCACGGAGNTAATATTCGCTCAGAGAATGCAGCAAAACCTCATACCTGTCTCTTATACACATCTGACGCTGCCGACGA
+
;@@DB?B;CFBB#2<:CB:FH<C@:<A?C::#1:86:BG9:8?8688?888EBF;783)=6-7=CC;ECD);?7;;>>AE;>(5;->AC@;B@;8?#####
@SRR1363257.77 GWZHISEQ01:153:C1W31ACXX:5:1101:5069:2307 length=101
ATAATAGGCAATCGCGTCGGAACAGTTACCGGCCAAAGAGAGGCAGGGACTTAACGGCATGATGGTGACCTCAGTTAAGAGAAGCCTGTCTCTTATACACA
+
+=?;:2,+A++AC:C:2@F6:CD:B09B?4)8@''8=))8=;=((5=4@?;@6;@?@BB;(535::>:>3(::(44:@::@3((9<32+::@(4@4+:>C3
@SRR1363257.78 GWZHISEQ01:153:C1W31ACXX:5:1101:5178:2440 length=101
ATATTAACAGTAGTATCAGTTATTTCTCTGATCTCTTTAGTCATTTGGGAGTCGACCTCAGAGAACCCGATTCTTGATCTCAGTTTGTTTAAGTCCCGTAA
+
BCCF

**Inspect reference FASTA file manually**

In [6]:
!zcat GCF_000005845.2_ASM584v2_genomic.fna.gz | head -20

>NC_000913.3 Escherichia coli str. K-12 substr. MG1655, complete genome
AGCTTTTCATTCTGACTGCAACGGGCAATATGTCTCTGTGTGGATTAAAAAAAGAGTGTCTGATAGCAGCTTCTGAACTG
GTTACCTGCCGTGAGTAAATTAAAATTTTATTGACTTAGGTCACTAAATACTTTAACCAATATAGGCATAGCGCACAGAC
AGATAAAAATTACAGAGTACACAACATCCATGAAACGCATTAGCACCACCATTACCACCACCATCACCATTACCACAGGT
AACGGTGCGGGCTGACGCGTACAGGAAACACAGAAAAAAGCCCGCACCTGACAGTGCGGGCTTTTTTTTTCGACCAAAGG
TAACGAGGTAACAACCATGCGAGTGTTGAAGTTCGGCGGTACATCAGTGGCAAATGCAGAACGTTTTCTGCGTGTTGCCG
ATATTCTGGAAAGCAATGCCAGGCAGGGGCAGGTGGCCACCGTCCTCTCTGCCCCCGCCAAAATCACCAACCACCTGGTG
GCGATGATTGAAAAAACCATTAGCGGCCAGGATGCTTTACCCAATATCAGCGATGCCGAACGTATTTTTGCCGAACTTTT
GACGGGACTCGCCGCCGCCCAGCCGGGGTTCCCGCTGGCGCAATTGAAAACTTTCGTCGATCAGGAATTTGCCCAAATAA
AACATGTCCTGCATGGCATTAGTTTGTTGGGGCAGTGCCCGGATAGCATCAACGCTGCGCTGATTTGCCGTGGCGAGAAA
ATGTCGATCGCCATTATGGCCGGCGTATTAGAAGCGCGCGGTCACAACGTTACTGTTATCGATCCGGTCGAAAAACTGCT
GGCAGTGGGGCATTACCTCGAATCTACCGTCGATATTGCTGAGTCCACCCGCCGTATTGCGGCAAGCCGCATTCCGGCTG
ATCACATGGTGCTGATGGCAGGTTTCACCGCCGGTAA

**Count reads in one fastq file**

In [8]:
!echo "initial_read_number_in_one_file=$(($(zcat amp_res_1.fastq.gz | wc -l)/4))"

initial_read_number_in_one_file=455876


## Inspect raw sequencing data with fastqc

**Install fastqc if it was not installed previously**

In [10]:
!sudo apt-get install fastqc

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  libapache-pom-java libcommons-compress-java libcommons-jexl2-java
  libcommons-logging-java libcommons-math3-java libcommons-parent-java
  libhtsjdk-java libjbzip2-java libngs-java libngs-sdk-dev libngs-sdk1
  libsnappy-java libsnappy-jni libxz-java
Suggested packages:
  libcommons-jexl2-java-doc libavalon-framework-java
  libcommons-logging-java-doc libexcalibur-logkit-java liblog4j1.2-java
  picard-tools
The following NEW packages will be installed:
  fastqc libapache-pom-java libcommons-compress-java libcommons-jexl2-java
  libcommons-logging-java libcommons-math3-java libcommons-parent-java
  libhtsjdk-java libjbzip2-java libngs-java libngs-sdk-dev libngs-sdk1
  libsnappy-java libsnappy-jni libxz

**Check if fastqc was installed correctly**

In [11]:
!fastqc -h


            FastQC - A high throughput sequence QC analysis tool

SYNOPSIS

	fastqc seqfile1 seqfile2 .. seqfileN

    fastqc [-o output dir] [--(no)extract] [-f fastq|bam|sam] 
           [-c contaminant file] seqfile1 .. seqfileN

DESCRIPTION

    FastQC reads a set of sequence files and produces from each one a quality
    control report consisting of a number of different modules, each one of 
    which will help to identify a different potential type of problem in your
    data.
    
    If no files to process are specified on the command line then the program
    will start as an interactive graphical application.  If files are provided
    on the command line then the program will run with no user interaction
    required.  In this mode it is suitable for inclusion into a standardised
    analysis pipeline.
    
    The options for the program as as follows:
    
    -h --help       Print this help file and exit
    
    -v --version    Print the version of the program and exit

**Run fastqc**

In [13]:
!fastqc -o . amp_res_1.fastq.gz amp_res_2.fastq.gz


Started analysis of amp_res_1.fastq.gz
Approx 5% complete for amp_res_1.fastq.gz
Approx 10% complete for amp_res_1.fastq.gz
Approx 15% complete for amp_res_1.fastq.gz
Approx 20% complete for amp_res_1.fastq.gz
Approx 25% complete for amp_res_1.fastq.gz
Approx 30% complete for amp_res_1.fastq.gz
Approx 35% complete for amp_res_1.fastq.gz
Approx 40% complete for amp_res_1.fastq.gz
Approx 45% complete for amp_res_1.fastq.gz
Approx 50% complete for amp_res_1.fastq.gz
Approx 55% complete for amp_res_1.fastq.gz
Approx 60% complete for amp_res_1.fastq.gz
Approx 65% complete for amp_res_1.fastq.gz
Approx 70% complete for amp_res_1.fastq.gz
Approx 75% complete for amp_res_1.fastq.gz
Approx 80% complete for amp_res_1.fastq.gz
Approx 85% complete for amp_res_1.fastq.gz
Approx 90% complete for amp_res_1.fastq.gz
Approx 95% complete for amp_res_1.fastq.gz
Analysis complete for amp_res_1.fastq.gz
Started analysis of amp_res_2.fastq.gz
Approx 5% complete for amp_res_2.fastq.gz
Approx 10% complete for

**Check out html files with reads quality assesment**

The number of reads in the basic statistics match what we calculated earlier (455876).

We can see unusual result in "Per base sequence quality" and "Per tile sequence quality" for amp_res_1.fastq.gz 
and one unusual result for amp_res_2.fastq.gz ("Per base sequence quality").

"Per base sequence quality" module raises a failure if the lower quartile for any base is less than 5 or if the median for any base is less than 20.
"Per tile sequence quality" raises a failure if there are significant transient problems such as bubbles going through the flowcell, or permanent problems such as smudges on the flowcell or debris inside the flowcell lane.
This module will issue a warning if any tile shows a mean Phred score more than 2 less than the mean for that base across all tiles.

## Filtering the reads

**Install trimmomatic**

In [14]:
!sudo apt-get install trimmomatic

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following NEW packages will be installed:
  trimmomatic
0 upgraded, 1 newly installed, 0 to remove and 27 not upgraded.
Need to get 749 kB of archives.
After this operation, 795 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 trimmomatic all 0.36+dfsg-3 [749 kB]
Fetched 749 kB in 1s (1,256 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg

**Run Trimmomatic in paired end mode, with following parameters:**\
Cut bases off the start of a read if quality below 20.\
Cut bases off the end of a read if quality below 20.\
Trim reads using a sliding window approach, with window size 10 and average quality within the window 20.\
Drop the read if it is below length 20.

In [15]:
!TrimmomaticPE -phred33 amp_res_1.fastq.gz amp_res_2.fastq.gz -baseout amp_res.fq.gz LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20


TrimmomaticPE: Started with arguments:
 -phred33 amp_res_1.fastq.gz amp_res_2.fastq.gz -baseout amp_res.fq.gz LEADING:20 TRAILING:20 SLIDINGWINDOW:10:20 MINLEN:20
Multiple cores found: Using 2 threads
Using templated Output files: amp_res_1P.fq.gz amp_res_1U.fq.gz amp_res_2P.fq.gz amp_res_2U.fq.gz
Input Read Pairs: 455876 Both Surviving: 446259 (97.89%) Forward Only Surviving: 9216 (2.02%) Reverse Only Surviving: 273 (0.06%) Dropped: 128 (0.03%)
TrimmomaticPE: Completed successfully


**Count Both Survived manually**

In [16]:
!echo "both_survived=$(($(zcat amp_res_1P.fq.gz | wc -l)/4))"

both_survived=446259


**Run fastqc on trimmed fastq files**

In [17]:
!fastqc -o . amp_res_1P.fq.gz amp_res_2P.fq.gz


Started analysis of amp_res_1P.fq.gz
Approx 5% complete for amp_res_1P.fq.gz
Approx 10% complete for amp_res_1P.fq.gz
Approx 15% complete for amp_res_1P.fq.gz
Approx 20% complete for amp_res_1P.fq.gz
Approx 25% complete for amp_res_1P.fq.gz
Approx 30% complete for amp_res_1P.fq.gz
Approx 35% complete for amp_res_1P.fq.gz
Approx 40% complete for amp_res_1P.fq.gz
Approx 45% complete for amp_res_1P.fq.gz
Approx 50% complete for amp_res_1P.fq.gz
Approx 55% complete for amp_res_1P.fq.gz
Approx 60% complete for amp_res_1P.fq.gz
Approx 65% complete for amp_res_1P.fq.gz
Approx 70% complete for amp_res_1P.fq.gz
Approx 75% complete for amp_res_1P.fq.gz
Approx 80% complete for amp_res_1P.fq.gz
Approx 85% complete for amp_res_1P.fq.gz
Approx 90% complete for amp_res_1P.fq.gz
Approx 95% complete for amp_res_1P.fq.gz
Analysis complete for amp_res_1P.fq.gz
Started analysis of amp_res_2P.fq.gz
Approx 5% complete for amp_res_2P.fq.gz
Approx 10% complete for amp_res_2P.fq.gz
Approx 15% complete for amp_

**Check out html files**\
We can see one red circle for amp_res_1P.fq.gz ("Per tile sequence quality") 
and no one for amp_res_2.fastq.gz.\
Sequence quality improved after trimming.\
However, Sequence Length Destribution turned to yellow for both files (this is natural and does not prevent us from continuing to analyze the data).

**Run Trimmomatic in paired end mode, with following parameters:**\
Cut bases off the start of a read if quality below 30.\
Cut bases off the end of a read if quality below 30.\
Trim reads using a sliding window approach, with window size 10 and average quality within the window 30.\
Drop the read if it is below length 30.

In [18]:
!TrimmomaticPE -phred33 amp_res_1.fastq.gz amp_res_2.fastq.gz -baseout amp_res2.fq.gz LEADING:30 TRAILING:30 SLIDINGWINDOW:10:30 MINLEN:30


TrimmomaticPE: Started with arguments:
 -phred33 amp_res_1.fastq.gz amp_res_2.fastq.gz -baseout amp_res2.fq.gz LEADING:30 TRAILING:30 SLIDINGWINDOW:10:30 MINLEN:30
Multiple cores found: Using 2 threads
Using templated Output files: amp_res2_1P.fq.gz amp_res2_1U.fq.gz amp_res2_2P.fq.gz amp_res2_2U.fq.gz
Input Read Pairs: 455876 Both Surviving: 360209 (79.01%) Forward Only Surviving: 36716 (8.05%) Reverse Only Surviving: 27291 (5.99%) Dropped: 31660 (6.94%)
TrimmomaticPE: Completed successfully


**We lost more then 20 % of reads, that can be a problem. 
Let's check fastqc result.**

In [19]:
!fastqc -o . amp_res2_1P.fq.gz amp_res2_2P.fq.gz

Started analysis of amp_res2_1P.fq.gz
Approx 5% complete for amp_res2_1P.fq.gz
Approx 10% complete for amp_res2_1P.fq.gz
Approx 15% complete for amp_res2_1P.fq.gz
Approx 20% complete for amp_res2_1P.fq.gz
Approx 25% complete for amp_res2_1P.fq.gz
Approx 30% complete for amp_res2_1P.fq.gz
Approx 35% complete for amp_res2_1P.fq.gz
Approx 40% complete for amp_res2_1P.fq.gz
Approx 45% complete for amp_res2_1P.fq.gz
Approx 50% complete for amp_res2_1P.fq.gz
Approx 55% complete for amp_res2_1P.fq.gz
Approx 60% complete for amp_res2_1P.fq.gz
Approx 65% complete for amp_res2_1P.fq.gz
Approx 70% complete for amp_res2_1P.fq.gz
Approx 75% complete for amp_res2_1P.fq.gz
Approx 80% complete for amp_res2_1P.fq.gz
Approx 85% complete for amp_res2_1P.fq.gz
Approx 90% complete for amp_res2_1P.fq.gz
Approx 95% complete for amp_res2_1P.fq.gz
Analysis complete for amp_res2_1P.fq.gz
Started analysis of amp_res2_2P.fq.gz
Approx 5% complete for amp_res2_2P.fq.gz
Approx 10% complete for amp_res2_2P.fq.gz
Appr

**Check out html files**\
We can see one red circle for amp_res2_1P.fq.gz ("Per tile sequence quality") 
and one for amp_res2_2.fastq.gz ("Per tile sequence quality"). It is a bit worse than previous variant. Besides, we do not want to lose a lot of reads - that would affect coverage. 

## Alignment


**Install aligner bwa**

In [20]:
!sudo apt-get install bwa

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
Suggested packages:
  samtools
The following NEW packages will be installed:
  bwa
0 upgraded, 1 newly installed, 0 to remove and 27 not upgraded.
Need to get 176 kB of archives.
After this operation, 404 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 bwa amd64 0.7.17-1ubuntu0.1 [176 kB]
Fetched 176 kB in 0s (450 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling ba

**Index the reference file**

In [21]:
!bwa index GCF_000005845.2_ASM584v2_genomic.fna.gz

[bwa_index] Pack FASTA... 0.06 sec
[bwa_index] Construct BWT for the packed sequence...
[bwa_index] 1.18 seconds elapse.
[bwa_index] Update BWT... 0.02 sec
[bwa_index] Pack forward-only FASTA... 0.04 sec
[bwa_index] Construct SA from BWT and Occ... 0.47 sec
[main] Version: 0.7.17-r1188
[main] CMD: bwa index GCF_000005845.2_ASM584v2_genomic.fna.gz
[main] Real time: 1.850 sec; CPU: 1.779 sec


**Alignment**

In [22]:
!bwa mem GCF_000005845.2_ASM584v2_genomic.fna.gz amp_res_1P.fq.gz amp_res_2P.fq.gz > alignment.sam

[M::bwa_idx_load_from_disk] read 0 ALT contigs
[M::process] read 106336 sequences (10000178 bp)...
[M::process] read 108264 sequences (10000141 bp)...
[M::mem_pestat] # candidate unique pairs for (FF, FR, RF, RR): (9, 51236, 0, 22)
[M::mem_pestat] skip orientation FF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation FR...
[M::mem_pestat] (25, 50, 75) percentile: (145, 185, 230)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 400)
[M::mem_pestat] mean and std.dev: (189.31, 63.57)
[M::mem_pestat] low and high boundaries for proper pairs: (1, 485)
[M::mem_pestat] skip orientation RF as there are not enough pairs
[M::mem_pestat] analyzing insert size distribution for orientation RR...
[M::mem_pestat] (25, 50, 75) percentile: (114, 158, 263)
[M::mem_pestat] low and high boundaries for computing mean and std.dev: (1, 561)
[M::mem_pestat] mean and std.dev: (131.82, 63.27)
[M::mem_pestat] low and high boundaries for pro

**Compress SAM file**

In [25]:
!sudo apt-get install samtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  cwltool libc-ares2 libhts2 libjs-bootstrap nodejs nodejs-doc
  python-asn1crypto python-avro python-cachecontrol python-certifi
  python-cffi-backend python-chardet python-cryptography python-enum34
  python-html5lib python-idna python-ipaddress python-isodate python-lockfile
  python-mistune python-openssl python-pkg-resources python-pyparsing
  python-rdflib python-rdflib-jsonld python-requests python-ruamel.yaml
  python-schema-salad python-shellescape python-six python-sparqlwrapper
  python-typing python-urllib3 python-webencodings
Suggested packages:
  python-cryptography-doc python-cryptography-vectors python-enum34-doc
  python-genshi python-lxml python-lockfile-doc python-openssl-doc
  pytho

In [26]:
!samtools view -S -b alignment.sam > alignment.bam

**Look at some summary**

In [27]:
!samtools flagstat alignment.bam

892776 + 0 in total (QC-passed reads + QC-failed reads)
0 + 0 secondary
258 + 0 supplementary
0 + 0 duplicates
891649 + 0 mapped (99.87% : N/A)
892518 + 0 paired in sequencing
446259 + 0 read1
446259 + 0 read2
888554 + 0 properly paired (99.56% : N/A)
890412 + 0 with itself and mate mapped
979 + 0 singletons (0.11% : N/A)
0 + 0 with mate mapped to a different chr
0 + 0 with mate mapped to a different chr (mapQ>=5)


**Sort and index BAM file**

In [28]:
!samtools sort alignment.bam -o alignment_sorted.bam
!samtools index alignment_sorted.bam

## Variant calling

**Create mpileup file**

In [30]:
!gunzip GCF_000005845.2_ASM584v2_genomic.fna

In [31]:
!samtools mpileup -f GCF_000005845.2_ASM584v2_genomic.fna alignment_sorted.bam >  my.mpileup

[mpileup] 1 samples in 1 input files
<mpileup> Set max per-file depth to 8000


**Check options in VarScan**

In [33]:
!java -jar VarScan.v2.4.0.jar mpileup2snp -h

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.2
Min avg qual:	15
P-value thresh:	0.01
USAGE: java -jar VarScan.jar mpileup2cns [pileup file] OPTIONS
	mpileup file - The SAMtools mpileup file

	OPTIONS:
	--min-coverage	Minimum read depth at a position to make a call [8]
	--min-reads2	Minimum supporting reads at a position to call variants [2]
	--min-avg-qual	Minimum base quality at a position to count a read [15]
	--min-var-freq	Minimum variant allele frequency threshold [0.01]
	--min-freq-for-hom	Minimum frequency to call homozygote [0.75]
	--p-value	Default p-value threshold for calling variants [99e-02]
	--strand-filter	Ignore variants with >90% support on one strand [1]
	--output-vcf	If set to 1, outputs in VCF format
	--vcf-sample-list	For VCF output, a list of sample names in order, one per line
	--variants	Report only variant (SNP/indel) positions [0]


**Variant calling**

In [34]:
!java -jar VarScan.v2.4.0.jar  mpileup2snp my.mpileup --min-var-freq 0.7 --variants --output-vcf 1 > VarScan_results.vcf

Only SNPs will be reported
Min coverage:	8
Min reads2:	2
Min var freq:	0.7
Min avg qual:	15
P-value thresh:	0.01
Reading input from my.mpileup
4641343 bases in pileup file
9 variant positions (6 SNP, 3 indel)
1 were failed by the strand-filter
5 variant positions reported (5 SNP, 0 indel)


Found 5 SNP.
Tried to lower the min-var-freq threshold to 50%, and got the same results.

## Automatic SNP annotation

Downloaded snpEff.jar file and GCF_000005845.2_ASM584v2_genomic.gbff.gz (sequence and annotation of our reference)

**Create snpEff.config**

In [35]:
!touch snpEff.config 
!echo "k12.genome : ecoli_K12" > snpEff.config 

**Create folder for the database**

In [36]:
!mkdir -p data/k12

**Put .gbk file there (unzip and rename to genes.gbk)**

In [37]:
!gunzip GCF_000005845.2_ASM584v2_genomic.gbff.gz
!cp GCF_000005845.2_ASM584v2_genomic.gbff data/k12/genes.gbk

**Create database**

In [41]:
!java -jar snpEff.jar build -genbank -v k12

00:00:00 SnpEff version SnpEff 5.1d (build 2022-04-19 15:49), by Pablo Cingolani
00:00:00 Command: 'build'
00:00:00 Building database for 'k12'
00:00:00 Reading configuration file 'snpEff.config'. Genome: 'k12'
00:00:00 Reading config file: /content/snpEff.config
00:00:00 done
00:00:01 Chromosome: 'NC_000913.3'	length: 4641652
00:00:01 Create exons from CDS (if needed): 
..................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

**Get annotations**

In [42]:
!java -jar snpEff.jar ann k12 VarScan_results.vcf > VarScan_results_annotated.vcf

In [43]:
!cat VarScan_results_annotated.vcf | grep -v "^#"

NC_000913.3	93043	.	C	G	.	PASS	ADP=17;WT=0;HET=0;HOM=1;NC=0;ANN=G|missense_variant|MODERATE|ftsI|b0084|transcript|b0084|protein_coding|1/1|c.1631C>G|p.Ala544Gly|1631/1767|1631/1767|544/588||,G|upstream_gene_variant|MODIFIER|murE|b0085|transcript|b0085|protein_coding||c.-123C>G|||||123|WARNING_TRANSCRIPT_NO_START_CODON,G|upstream_gene_variant|MODIFIER|murF|b0086|transcript|b0086|protein_coding||c.-1607C>G|||||1607|,G|upstream_gene_variant|MODIFIER|mraY|b0087|transcript|b0087|protein_coding||c.-2959C>G|||||2959|,G|upstream_gene_variant|MODIFIER|murD|b0088|transcript|b0088|protein_coding||c.-4044C>G|||||4044|,G|downstream_gene_variant|MODIFIER|cra|b0080|transcript|b0080|protein_coding||c.*4011C>G|||||4011|WARNING_TRANSCRIPT_NO_START_CODON,G|downstream_gene_variant|MODIFIER|mraZ|b0081|transcript|b0081|protein_coding||c.*2951C>G|||||2951|,G|downstream_gene_variant|MODIFIER|rsmH|b0082|transcript|b0082|protein_coding||c.*2008C>G|||||2008|,G|downstream_gene_variant|MODIFIER|ftsL|b0083|transcri